In [2]:
import numpy as np
import torch

# PyTorch テンソルの表示オプションを設定することで、
# print 時の視認性と簡潔さを向上させる

torch.set_printoptions(
    edgeitems=2,  # 各次元の先頭と末尾の要素数（中間は "..." で省略）
    precision=2,  # 浮動小数点数の小数点以下の桁数（例: 0.123456 → 0.12）
    linewidth=75,  # 1行あたりの最大表示幅（長すぎる行は改行される）
)

In [3]:
import csv
import numpy as np

# ワイン品質データセット（白ワイン）のCSVファイルのパス
wine_path = "../../data/p1ch4/tabular-wine/winequality-white.csv"

# CSVファイルをNumPy配列として読み込む
# - dtype=np.float32: すべての値をfloat32型として読み込む
# - delimiter=";": 区切り文字はセミコロン（このCSVはExcel形式）
# - skiprows=1: ヘッダー行（列名）をスキップ
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)

# 読み込んだデータを表示（shapeや内容を確認）
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]],
      shape=(4898, 12), dtype=float32)

In [4]:
# csv.reader でファイルを開き、最初の行（=列名の行）だけを取得
col_list = next(csv.reader(open(wine_path), delimiter=";"))

# データ本体はすでに NumPy 配列として読み込まれていると仮定
# wineq_numpy = np.loadtxt(...)

# データの形状と列名一覧を確認
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [5]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [6]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [7]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [8]:
target = wineq[:, -1].long()
target

tensor([6, 6,  ..., 7, 6])

In [9]:
# 例: target = tensor([3, 1, 4, 0, ...]) のような整数クラスラベル（0〜9）

# ワンホット形式を格納するテンソルを初期化（全ゼロ）
# shape: [サンプル数, クラス数] → 10クラス分類タスクを想定
target_onehot = torch.zeros(target.shape[0], 10)

# target.unsqueeze(1) によって shape を [N, 1] に拡張し、
# scatter_ により 1.0 を指定インデックスに代入（in-place操作）
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

In [10]:
# 例: target = tensor([2, 0, 4]) のような shape: [N]（1次元ベクトル）を仮定

# 次元1（列方向）に1次元を追加し、shape を [N, 1] にする
# これは scatter_ や loss 計算（CrossEntropyLossなど）で必要になる形式
target_unsqueezed = target.unsqueeze(1)

# 確認
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

In [12]:
# data: shape = [N, D] （N個のデータサンプル、D個の特徴量）

# 各特徴量（列）ごとに平均値を計算 → shape: [D]
# dim=0 は「行方向に沿って」集約（= 各列ごと）
data_mean = torch.mean(data, dim=0)

# 計算結果の確認
data_mean

tensor([6.85e+00, 2.78e-01, 3.34e-01, 6.39e+00, 4.58e-02, 3.53e+01,
        1.38e+02, 9.94e-01, 3.19e+00, 4.90e-01, 1.05e+01])

In [14]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.12e-01, 1.02e-02, 1.46e-02, 2.57e+01, 4.77e-04, 2.89e+02,
        1.81e+03, 8.95e-06, 2.28e-02, 1.30e-02, 1.51e+00])

In [15]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.72e-01, -8.18e-02,  ..., -3.49e-01, -1.39e+00],
        [-6.57e-01,  2.16e-01,  ...,  1.34e-03, -8.24e-01],
        ...,
        [-1.61e+00,  1.17e-01,  ..., -9.63e-01,  1.86e+00],
        [-1.01e+00, -6.77e-01,  ..., -1.49e+00,  1.04e+00]])

In [ ]:
# target: shape = [N] の整数ラベルテンソル（例：0〜9のクラスラベル）

# 条件：クラスラベルが3以下の要素をTrueとするブールマスクを作成
bad_indexes = target <= 3

# 各種情報を確認
bad_indexes.shape  # 元の target と同じ shape → (N,)
bad_indexes.dtype  # torch.bool（真偽値マスク）
bad_indexes.sum()  # 条件に合致する要素の個数（Trueの数）を整数で返す

(torch.Size([4898]), torch.bool, tensor(20))

In [17]:
# bad_indexes: shape = [N] の torch.bool 型テンソル（例：target <= 3 の結果）
# data: shape = [N, D] の特徴量テンソル（Nサンプル、D特徴量）

# 条件に合致する行（bad_indexes が True の行）だけを抽出
bad_data = data[bad_indexes]

# 抽出後の shape を確認
bad_data.shape  # → (n_bad, D)

torch.Size([20, 11])

In [18]:
# 条件でターゲット（品質スコア）を分類し、それぞれの特徴量の平均を求める

# 品質スコアが低い（0〜3）データ
bad_data = data[target <= 3]

# 品質スコアが中間（4〜6）にあるデータ <1>
mid_data = data[(target > 3) & (target < 7)]

# 品質スコアが高い（7〜9など）データ
good_data = data[target >= 7]

# 各クラスの平均ベクトルを計算（dim=0で列方向＝各特徴量）
bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

# 各特徴量の名前と、bad/mid/good グループでの平均値を並べて出力
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print("{:2} {:20} {:6.2f} {:6.2f} {:6.2f}".format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [19]:
# しきい値を設定（特徴量 'total sulfur dioxide' の平均など）
total_sulfur_threshold = 141.83

# 7番目の特徴量（列 index = 6）は 'total sulfur dioxide' として想定
total_sulfur_data = data[:, 6]  # shape: [N]

# この特徴量がしきい値未満のサンプルを「予測が良い」と仮定
# torch.lt = less than（要素ごとの比較） → ブールテンソルが返る
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

# 出力の確認
predicted_indexes.shape  # → (N,) 元データと同じ行数
predicted_indexes.dtype  # → torch.bool
predicted_indexes.sum()  # → True の数（しきい値を下回ったサンプル数）

tensor(2727)

In [20]:
# target: ワインの品質スコア（整数ラベル）を含むテンソル [N]

# 実際に「良いワイン」とみなす条件（スコアが6より大きい）
actual_indexes = target > 5

# 出力情報の確認
actual_indexes.shape  # → [N] 元のデータと同じ行数
actual_indexes.dtype  # → torch.bool
actual_indexes.sum()  # → Trueの個数（= 実際に良いワインの数）

tensor(3258)

In [21]:
# predicted_indexes: モデルやしきい値による「良いワイン」と予測したサンプルのブールマスク
# actual_indexes: 実際に品質スコア > 5 の「良いワイン」のブールマスク

# 予測と実際が一致しているサンプル数（AND を取る）
n_matches = torch.sum(actual_indexes & predicted_indexes).item()

# 「良いワイン」と予測された総数（＝陽性予測数）
n_predicted = torch.sum(predicted_indexes).item()

# 実際の「良いワイン」の総数（＝真の陽性数 + 偽陰性数）
n_actual = torch.sum(actual_indexes).item()

# 出力：一致数、Precision（適合率）、Recall（再現率）
n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)